### Recommendations with MovieTweetings: Most Popular Recommendation

Now that you have created the necessary columns we will be using throughout the rest of the lesson on creating recommendations, let's get started with the first of our recommendations.

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

In [93]:
pd.set_option("display.max_columns", None)

#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [3]:
movies.head(2)

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [4]:
reviews.head(2)

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [128]:
my_movies = movies.merge(reviews, on='movie_id', how='left').sort_values('movie_id')

In [130]:
my_movies.head(2)

,movie_id,movie,genre,date_x,1800's,1900's,2000's,History,News,Horror,Musical,Film-Noir,Mystery,Adventure,Sport,War,Music,Reality-TV,Adult,Crime,Family,Drama,Talk-Show,Biography,Sci-Fi,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller,user_id,rating,timestamp,date_y,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,40425,5,1396981211,2014-04-08 18:20:11,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,48654,10,1412878553,2014-10-09 18:15:53,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0


In [131]:
my_movies[my_movies['date_x'].isin(['1902', '1903'])].head(2)

,movie_id,movie,genre,date_x,1800's,1900's,2000's,History,News,Horror,Musical,Film-Noir,Mystery,Adventure,Sport,War,Music,Reality-TV,Adult,Crime,Family,Drama,Talk-Show,Biography,Sci-Fi,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller,user_id,rating,timestamp,date_y,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
23,417,Le voyage dans la lune (1902),Short|Adventure|Fantasy,1902,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,52959,7,1530047326,2018-06-26 21:08:46,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
22,417,Le voyage dans la lune (1902),Short|Adventure|Fantasy,1902,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,50597,6,1433527435,2015-06-05 18:03:55,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [133]:
my_movies[pd.DataFrame(my_movies['genre'].tolist()).isin(['Adventure']).any(1).values]

,movie_id,movie,genre,date_x,1800's,1900's,2000's,History,News,Horror,Musical,Film-Noir,Mystery,Adventure,Sport,War,Music,Reality-TV,Adult,Crime,Family,Drama,Talk-Show,Biography,Sci-Fi,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller,user_id,rating,timestamp,date_y,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
4893,39743,Queen of the Amazons (1947),Adventure,1947,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9864,7,1453789418,2016-01-26 06:23:38,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4894,39743,Queen of the Amazons (1947),Adventure,1947,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22915,5,1377843999,2013-08-30 06:26:39,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
6612,45053,The Prisoner of Zenda (1952),Adventure,1952,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,29040,9,1399216842,2014-05-04 15:20:42,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
6817,45617,Cet homme est dangereux (1953),Adventure,1953,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3844,4,1456226951,2016-02-23 11:29:11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
8115,47669,West of Zanzibar (1954),Adventure,1954,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6577,4,1368857936,2013-05-18 06:18:56,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
710607,6673840,Vremya Pervyh (2017),Adventure,2017,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,18019,6,1496579204,2017-06-04 12:26:44,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
710604,6673840,Vremya Pervyh (2017),Adventure,2017,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6792,7,1511042960,2017-11-18 22:09:20,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
710605,6673840,Vremya Pervyh (2017),Adventure,2017,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7767,9,1507803497,2017-10-12 10:18:17,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
710606,6673840,Vremya Pervyh (2017),Adventure,2017,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12376,5,1506788991,2017-09-30 16:29:51,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [185]:
check = my_movies['genre'].str.split('|')
check

0         [Documentary, Short]
1         [Documentary, Short]
2         [Documentary, Short]
3                          NaN
4              [Short, Horror]
                  ...         
712332                [Comedy]
712333                [Comedy]
712334                [Comedy]
712335                [Comedy]
712336                 [Drama]
Name: genre, Length: 712337, dtype: object

In [187]:
check.loc[check.isnull()] = check.loc[check.isnull()].apply(lambda x: [])
check

0         [Documentary, Short]
1         [Documentary, Short]
2         [Documentary, Short]
3                           []
4              [Short, Horror]
                  ...         
712332                [Comedy]
712333                [Comedy]
712334                [Comedy]
712335                [Comedy]
712336                 [Drama]
Name: genre, Length: 712337, dtype: object

In [191]:
my_movies[pd.DataFrame(check.tolist()).isin(['Adventure']).any(1).values].head(2)

,movie_id,movie,genre,date_x,1800's,1900's,2000's,History,News,Horror,Musical,Film-Noir,Mystery,Adventure,Sport,War,Music,Reality-TV,Adult,Crime,Family,Drama,Talk-Show,Biography,Sci-Fi,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller,user_id,rating,timestamp,date_y,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
23,417,Le voyage dans la lune (1902),Short|Adventure|Fantasy,1902,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,52959,7,1530047326,2018-06-26 21:08:46,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
22,417,Le voyage dans la lune (1902),Short|Adventure|Fantasy,1902,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,50597,6,1433527435,2015-06-05 18:03:55,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [ ]:
for x in range(my_movies[0]):
    if my_movies.genre = 1:
        

In [102]:
def popular_recommendations(user_id, n_top):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Do stuff
    # create combined set
    my_movies = movies.merge(reviews, on='movie_id', how='left')
    
    # create a dataset of movies that have at least 5 ratings
    movies_by_nr_ratings = my_movies[['movie_id', 'rating']].groupby('movie_id').count()
    movies_by_nr_ratings = movies_by_nr_ratings[movies_by_nr_ratings['rating']>=5]
    
    # create a dataset of movies based on the average rating
    movies_avg_ratings = my_movies[['movie_id', 'rating']].groupby('movie_id').mean()
    
    # get the last rating by movie
    latest_review_by_movie = my_movies[['movie_id', 'date_y']].groupby('movie_id').max()
    
    # starting point are movies that have at least 5 ratings. take movies_by_nr_ratings and left join the other datasets on it
    final_df_movies = movies_by_nr_ratings.merge(movies_avg_ratings, on='movie_id', how='left', suffixes=('_nr', '_avg'))
    final_df_movies = final_df_movies.merge(latest_review_by_movie, on='movie_id', how='left')
    final_df_movies = final_df_movies.merge(movies[['movie_id', 'movie']], on='movie_id', how='left').reset_index()
    
    # sort by average rating, number of ratings, lastest rating
    final_df_movies = final_df_movies.sort_values(['rating_avg', 'rating_nr', 'date_y'], ascending=[False, False, False])
    
    # clasify top_movies based on the input number n_top
    top_movies = final_df_movies[:n_top]['movie'].to_list()
    
    return top_movies # a list of the n_top movies as recommended

In [29]:
movies_by_nr_ratings = reviews[['movie_id', 'rating']].groupby('movie_id').count()
movies_by_nr_ratings = movies_by_nr_ratings[movies_by_nr_ratings['rating']>=5].sort_values('rating')
movies_by_nr_ratings.head(2)

,rating
movie_id,
8075496,5
5181852,5


In [20]:
movies_avg_ratings = reviews[['movie_id', 'rating']].groupby('movie_id').mean()
movies_avg_ratings.sort_values('rating', ascending=False).head(2)

,rating
movie_id,
105107,10.0
1684917,10.0


In [48]:
latest_review_by_movie = reviews[['movie_id', 'date']].groupby('movie_id').max()
latest_review_by_movie.head(2)

,date
movie_id,
8,2014-04-08 18:20:11
10,2014-10-09 18:15:53


In [30]:
movies_by_nr_ratings.shape

(9974, 1)

In [31]:
movies_avg_ratings.shape

(31245, 1)

In [79]:
# left join
final_list = movies_by_nr_ratings.merge(movies_avg_ratings, on='movie_id', how='left', suffixes=('_nr', '_avg'))
final_list = final_list.merge(latest_review_by_movie, on='movie_id', how='left')
final_list = final_list.merge(movies[['movie_id', 'movie']], on='movie_id', how='left').reset_index()
final_list.head(2)

,index,movie_id,rating_nr,rating_avg,date,movie
0,0,8075496,5,7.0,2018-06-21 14:34:27,Bao (2018)
1,1,5181852,5,3.8,2018-06-21 13:43:53,Slumber (2017)


Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [103]:
# Put your solutions for each of the cases here

# Top 20 movies recommended for id 1

recs_20_for_1 = popular_recommendations(1, 20) # Your solution list here

# Top 5 movies recommended for id 53968
recs_5_for_53968 = popular_recommendations(53968, 5) # Your solution list here

# Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations(70000, 100) # Your solution list here

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations(43, 35) # Your solution list here



In [104]:
### You Should Not Need To Modify Anything In This Cell
ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast

# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

If you got here, looks like you are good to go!  Nice job!


**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
```

In [211]:
def popular_recommendations_by_year_genre(user_id, n_top, years=[], genres=[]):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Do stuff
    # create combined set
    my_movies = movies.merge(reviews, on='movie_id', how='left')
    
    # apply filter for defined list of years
    my_movies = my_movies[my_movies['date_x'].isin(years)]
    
    # apply fitler for defined list of genres
    # split genre column into separate lists
    df_genres = my_movies['genre'].str.split('|')
    # replace nan with empty lists so we can use newly created list for filtering
    df_genres.loc[df_genres.isnull()] = df_genres.loc[df_genres.isnull()].apply(lambda x: [])
    # filter only for movies where movie genre is in the pre-defined list
    my_movies = my_movies[pd.DataFrame(df_genres.tolist()).isin(genres).any(1).values]
    
    # create a dataset of movies that have at least 5 ratings
    movies_by_nr_ratings = my_movies[['movie_id', 'rating']].groupby('movie_id').count()
    movies_by_nr_ratings = movies_by_nr_ratings[movies_by_nr_ratings['rating']>=5]
    
    # create a dataset of movies based on the average rating
    movies_avg_ratings = my_movies[['movie_id', 'rating']].groupby('movie_id').mean()
    
    # get the last rating by movie
    latest_review_by_movie = my_movies[['movie_id', 'date_y']].groupby('movie_id').max()
    
    # starting point are movies that have at least 5 ratings. take movies_by_nr_ratings and left join the other datasets on it
    final_df_movies = movies_by_nr_ratings.merge(movies_avg_ratings, on='movie_id', how='left', suffixes=('_nr', '_avg'))
    final_df_movies = final_df_movies.merge(latest_review_by_movie, on='movie_id', how='left')
    final_df_movies = final_df_movies.merge(movies[['movie_id', 'movie']], on='movie_id', how='left').reset_index()
    
    # sort by average rating, number of ratings, lastest rating
    final_df_movies = final_df_movies.sort_values(['rating_avg', 'rating_nr', 'date_y'], ascending=[False, False, False])
    
    # clasify top_movies based on the input number n_top
    top_movies = final_df_movies[:n_top]['movie'].to_list()
    
    return top_movies # a list of the n_top movies as recommended

In [209]:
popular_recommendations_by_year_genre('1', 20, years=['2015', '2016', '2017', '2018'], genres=['History'])

['Ayla: The Daughter of War (2017)',
 'I Believe in Miracles (2015)',
 'The Farthest (2017)',
 'Sado (2015)',
 'Hatred (2016)',
 'Kincsem (2017)',
 'Nise - O Coração da Loucura (2015)',
 'LA 92 (2017)',
 'Straight Outta Compton (2015)',
 'Manjhi: The Mountain Man (2015)',
 'Only the Dead (2015)',
 'Spotlight (2015)',
 'Under sandet (2015)',
 'Airlift (2016)',
 'Dunkirk (2017)',
 'Taeksi Woonjunsa (2017)',
 'The Battleship Island (2017)',
 'Darkest Hour (2017)',
 'Best of Enemies (2015)',
 'The Ghazi Attack (2017)']

In [210]:
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])

['Ayla: The Daughter of War (2017)',
 'I Believe in Miracles (2015)',
 'The Farthest (2017)',
 'Sado (2015)',
 'Hatred (2016)',
 'Kincsem (2017)',
 'Nise - O Coração da Loucura (2015)',
 'LA 92 (2017)',
 'Straight Outta Compton (2015)',
 'Manjhi: The Mountain Man (2015)',
 'Only the Dead (2015)',
 'Spotlight (2015)',
 'Under sandet (2015)',
 'Airlift (2016)',
 'Dunkirk (2017)',
 'Taeksi Woonjunsa (2017)',
 'The Battleship Island (2017)',
 'Darkest Hour (2017)',
 'Best of Enemies (2015)',
 'The Ghazi Attack (2017)']